In [2]:
from pyspark.sql import SparkSession 
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
spark = SparkSession.builder.appName("Linear regression model").getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
data.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [10]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [11]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [13]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [14]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [15]:
assembler = VectorAssembler(inputCols = ['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [16]:
output = assembler.transform(data)

In [17]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [19]:
final_data = output.select('features','Yearly Amount Spent')

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [22]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                355|
|   mean|  497.0910821579027|
| stddev|  76.65532342486172|
|    min| 256.67058229005585|
|    max|  712.3963268096637|
+-------+-------------------+



In [23]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                145|
|   mean|  504.7564480223448|
| stddev|  85.51151907687262|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [24]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [25]:
lr_model = lr.fit(train_data)

In [26]:
test_results = lr_model.evaluate(test_data)

In [27]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-16.458621443674133|
| -6.896694649140329|
|-0.9670549519549922|
|  19.41014318466324|
|  3.411781431248187|
|  3.871199869622046|
|-5.5401042499646564|
| -8.393750296598853|
| -4.385219371704636|
|-26.735377261546887|
| -6.947645487352361|
|-1.3673787660642915|
| -6.894933146703295|
| -11.99611742356069|
|  7.868711909264164|
| -9.206510948226821|
|  7.788676050697234|
| 10.867048006663822|
| 12.423671093470432|
|-13.654098318055958|
+-------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

9.78817568078591

In [29]:
test_results.r2

0.9868065073890578

In [30]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [32]:
unlabeled_data = test_data.select('features')

In [33]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.8162006488763...|
|[31.0613251567161...|
|[31.2606468698795...|
|[31.3123495994443...|
|[31.3584771924370...|
|[31.3662121671876...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.6253601348306...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.9096268275227...|
|[31.9262720263601...|
|[31.9365486184489...|
+--------------------+
only showing top 20 rows



In [35]:
predictions = lr_model.transform(unlabeled_data)

In [36]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8162006488763...|282.54496239214313|
|[31.0613251567161...|494.45215270704193|
|[31.2606468698795...|422.29368620890637|
|[31.3123495994443...| 444.1812748432774|
|[31.3584771924370...| 491.7641690182272|
|[31.3662121671876...| 426.7176826868629|
|[31.5257524169682...|449.50573105984654|
|[31.5261978982398...|417.48827648893666|
|[31.6253601348306...|380.72212012862883|
|[31.6739155032749...| 502.4604451714281|
|[31.7207699002873...| 545.7225789653753|
|[31.7216523605090...|349.14430539793693|
|[31.7242025238451...| 510.2828204346638|
|[31.8093003166791...| 548.7680167864019|
|[31.8209982016720...| 416.8065691039492|
|[31.8279790554652...|449.20925849516834|
|[31.8512531286083...|465.20357061610116|
|[31.9096268275227...| 552.5789876665754|
|[31.9262720263601...|379.78126235085597|
|[31.9365486184489...|440.85348321338415|
+--------------------+------------

In [37]:
# Linear Regression model

df = spark.read.csv("cruise_ship_info.csv", inferSchema=True, header=True)

In [38]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [39]:
for ship in df.head(5):
    print(ship)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)




In [41]:
df.groupBy('cruise_line').count().show()

+-----------------+-----+
|      cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [42]:
from pyspark.ml.feature import StringIndexer

In [43]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_cat')
indexed = indexer.fit(df).transform(df)

In [46]:
indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_cat=1.0)]

In [50]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [51]:
indexed.columns


['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat']

In [52]:
assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat'], outputCol = 'features')

In [53]:
output = assembler.transform(indexed)

In [55]:
output.select('features', 'crew').show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [56]:
final_data = output.select('features','crew')

In [57]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [58]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              113|
|   mean|7.945752212389394|
| stddev|3.519526095663038|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [59]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                45|
|   mean| 7.413555555555557|
| stddev|3.4726759105620526|
|    min|              0.59|
|    max|             13.13|
+-------+------------------+



In [60]:
from pyspark.ml.regression import LinearRegression

In [61]:
ship_lr = LinearRegression(labelCol='crew')

In [62]:
trained_ship_model = ship_lr.fit(train_data)

In [63]:
ship_results = trained_ship_model.evaluate(test_data)

In [64]:
ship_results.rootMeanSquaredError

7.046977339527127e-16

In [65]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              113|
|   mean|7.945752212389394|
| stddev|3.519526095663038|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [66]:
ship_results.r2

1.0

In [67]:
ship_results.meanSquaredError

4.965988962380883e-31

In [68]:
ship_results.meanAbsoluteError

4.268190739114491e-16

In [69]:
from pyspark.sql.functions import corr

In [70]:
df.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [71]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [72]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

